In [ ]:
#! pip install python-louvain
#! pip install python-igraph
#! pip install numpy pandas scikit-learn python-igraph
#! pip install --upgrade python-igraph
#! pip uninstall igraph


In [ ]:
from dfply import *
from IPython.display import display
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine, event

conn_str = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=KVHSQLPC56;DATABASE=AHDA;Trusted_Connection=yes;'
conn_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_str})
engine = create_engine(conn_url)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from collections import defaultdict
from community import community_louvain
from sqlalchemy import create_engine
from igraph import Graph
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import igraph
import random

class PatientCommunityAnalysis:
    def __init__(self, engine):
        self.engine = engine
        self.graph2 = None
    
    def interactions(self):
    #add your own query here. For example, for falls, you need to retrieve interactions for all unique patients 
    #using HF.interactions.
        query_dt = ''' 
            WITH FirstFall AS (
       SELECT 
              Distinct patient_id
              ,min([admit_datetime]) as first_fall
              ,min(diagnosis_age) as first_fall_age
       FROM 
              Core.acute_care_diagnoses acd
       WHERE 
              diagnosis_type_code NOT IN ('3', '4', '0')
              AND SUBSTRING(icd10_code, 1, 3) IN ('W06', 'W07', 'W08', 'W13', 'W14', 'W15', 'W16', 'W17', 
                                      'X80', 'Y01', 'Y30', 'W00', 'W01', 'W03', 'W04', 'W18', 
                                      'W10', 'W05')
              AND diagnosis_age >= 65
       GROUP BY 
              patient_id
)
SELECT 
       i.patient_id, i.service_class_id, i.start_datetime, i.end_datetime
FROM 
       FirstFall
INNER JOIN 
       HF.interactions i ON i.patient_id = FirstFall.patient_id
WHERE 
       i.start_datetime < first_fall
AND        
        i.service_class_id NOT IN (80,150,152,68)
ORDER BY i.patient_id, i.start_datetime
        '''
        df = pd.read_sql(query_dt, self.engine)
        interactions = df[df['service_class_id'].notna()]
        #interactions = interactions.dropna() 
        unique_patients = interactions['patient_id'].unique()
       
        train_set = interactions[interactions['patient_id'].isin(unique_patients)]
        return train_set
  
    def sequence(self, interactions_df):
        df = interactions_df
        df = df.astype({"service_class_id": str}, errors='raise')
        df4 = df.groupby('patient_id')['service_class_id'].agg(' '.join).reset_index()
        return df4
        
    def patients_projection(self, interactions_df):
        df = interactions_df.copy()
        print(df)

        # Ensure patient IDs are strings (if necessary)
        df['patient_id'] = df['patient_id'].astype(str)

        # **Consistent Patient ID Mapping**
        # Sort patient IDs to ensure consistent ordering
        unique_patient_ids = sorted(df['patient_id'].unique())
        self.patient_id_to_int = {pat_id: idx for idx, pat_id in enumerate(unique_patient_ids)}
        self.int_to_patient_id = {idx: pat_id for pat_id, idx in self.patient_id_to_int.items()}

        # Map patient IDs to integer indices
        df['patient_id_int'] = df['patient_id'].map(self.patient_id_to_int)

        result_map = defaultdict(dict)

        # Loop over each service and find patients who share the same service
        svc_unique = df['service_class_id'].unique()

        for svc in svc_unique:
            index = df.index[df['service_class_id'] == svc].tolist()
            patients = df.loc[index, 'patient_id_int'].unique()

            for i in range(len(patients)):
                for j in range(i + 1, len(patients)):
                    p1, p2 = patients[i], patients[j]
                    if p1 < p2:
                        result_map[p1][p2] = result_map[p1].get(p2, 0) + 1
                    else:
                        result_map[p2][p1] = result_map[p2].get(p1, 0) + 1

        # Convert the results into a DataFrame with source, target, and weight (shared services)
        source, target, weight = [], [], []
        for key, neighbors in result_map.items():
            for n, w in neighbors.items():
                source.append(key)
                target.append(n)
                weight.append(w)

        df3 = pd.DataFrame({'source': source, 'target': target, 'weight': weight})
        return df3

    def create_graph(self, interactions_df):
        # Create patient projection based on shared services
        df = self.patients_projection(interactions_df)

        # Create an igraph graph using the edges and weights
        edges = list(zip(df['source'], df['target']))
        weights = df['weight'].tolist()

        # Create the graph with the correct number of vertices
        num_vertices = len(self.patient_id_to_int)
        graph = igraph.Graph(n=num_vertices, edges=edges, directed=False)
        graph.es['weight'] = weights

        # Set the vertex names to patient IDs
        graph.vs['name'] = [self.int_to_patient_id[idx] for idx in range(graph.vcount())]
        
        self.graph2 = graph
        
        self.map4 = {idx: pat_id for idx, pat_id in enumerate([self.int_to_patient_id[idx] for idx in range(self.graph2.vcount())])}

        self.graph2 = graph

    def community_detection_on_patients(self, number_of_iterations=-1):
        numberofitterations=number_of_iterations
        # first itteration
        louvain = self.graph2.community_multilevel(weights=self.graph2.es['weight'], return_levels=False)
        pi = []
        piI = []

        numberofclustures = 0
        for i in range(len(louvain)):
            component = louvain[i]

            if (len(component) > 1):
                piI.append(component)

        pi.append(piI)
        itteration = 1
        if numberofitterations==-1:
            while True:
                piI1 = []
                Previous_components = pi[itteration - 1]
                for community in pi[itteration - 1]:

                    # print(community)
                    induced_subgraph = self.graph2.induced_subgraph(list(community))
                    louvain1 = induced_subgraph.community_multilevel(weights=induced_subgraph.es['weight'], return_levels=False)
                    for i in range(len(louvain1)):
                        subgraph = louvain1[i]
                        # print(subgraph)
                        lst = []
                        for maped_vertex in subgraph:
                            map = community[maped_vertex]
                            lst.append(map)
                        # print(lst)
                        # print(tuple(lst))

                        piI1.append((lst))
                pi.append(piI1)
                current_components = pi[itteration]
                if not current_components > Previous_components:
                    break
                else:
                    itteration += 1
        if numberofitterations!=-1:
            if numberofitterations == -1:
                while True:
                    piI1 = []
                    Previous_components = pi[itteration - 1]
                    for community in pi[itteration - 1]:

                        # print(community)
                        induced_subgraph = self.graph2.induced_subgraph(list(community))
                        louvain1 = induced_subgraph.community_multilevel(weights=induced_subgraph.es['weight'],
                                                                         return_levels=False)
                        for i in range(len(louvain1)):
                            subgraph = louvain1[i]
                            # print(subgraph)
                            lst = []
                            for maped_vertex in subgraph:
                                map = community[maped_vertex]
                                lst.append(map)
                            # print(lst)
                            # print(tuple(lst))

                            piI1.append((lst))
                    pi.append(piI1)
                    current_components = pi[itteration]
                    if not current_components > Previous_components:
                        break
                    else:
                        itteration += 1


            if numberofitterations != -1:
                while itteration<numberofitterations:
                    piI1 = []
                    Previous_components = pi[itteration - 1]
                    for community in pi[itteration - 1]:

                        # print(community)
                        induced_subgraph = self.graph2.induced_subgraph(list(community))
                        louvain1 = induced_subgraph.community_multilevel(weights=induced_subgraph.es['weight'],
                                                                         return_levels=False)
                        for i in range(len(louvain1)):
                            subgraph = louvain1[i]
                            # print(subgraph)
                            lst = []
                            for maped_vertex in subgraph:
                                map = community[maped_vertex]
                                lst.append(map)
                            # print(lst)
                            # print(tuple(lst))

                            piI1.append((lst))
                    pi.append(piI1)
                    current_components = pi[itteration]
                    if not current_components > Previous_components:
                        break
                    else:
                        itteration += 1


        
        print("number of all iterations",len(pi))
        for i in range(len(pi)):
            iteration=pi[i]
            print("number of commmunities in "+str(i+1)+' '+"iteration",len(iteration ))
      
    
       #prevent calling community detection for patients in create table function:
       #return pi,map1,map2
        self.pi6=pi
        
        
        return self.pi6,self.graph2  
    
    def create_table_for_patients(self,iteration2=-1):
        itteration2=iteration2
        if itteration2==-1:
            itteration=len(self.pi6)
            print(itteration)
        if itteration2 !=-1:
            itteration=itteration2
        weight = []
        out_degree=[]
        weightd_degree=[]
        for j in range(len(self.pi6)):
            itteration1 = self.pi6[j]

            weightI = []
            out_degreeI = list()
            weightd_degreeI=[]
            for i in range(len(itteration1)):
                component = itteration1[i]

                subgraph = self.graph2.induced_subgraph(component)
                total_degree = self.graph2.strength(component, mode='all', loops=True, weights="weight")
                weightd_degreeI.append(total_degree)
                lst = list(range(len(component)))
                in_degree = subgraph.strength(lst, mode='all', loops=True, weights="weight")
                weightI.append(in_degree)
                lst2=[]
                for item1, item2 in zip(total_degree,in_degree ):
                    item = item1 - item2
                    lst2.append(item)
                out_degreeI.append(lst2) 

            weight.append(weightI)
            out_degree.append(out_degreeI)
            weightd_degree.append(weightd_degreeI)


        if itteration2==-1:
            itteration=len(self.pi6)
            #for community id
            
            self.vertices = []
            idd = 1
            id = []
            age=[]
            gender=[]
            primary=[]
            fsa=[]
            n_encounters=[]
            n_interactions=[]
            first_admit_dt=[]
            last_admit_dt=[]
            covid_positive=[]            
            
            for component in self.pi6[itteration - 1]:
                for vertex in component:
                    v= self.map4.get(vertex)
                    self.vertices.append(v)
                    id.append(idd)
                idd = idd + 1
            #for weight
            weightt=[]
            for component in weight[itteration - 1]:
                for degree in component:
                    weightt.append(degree)

            #for out degree       
            out_degree2=[]   
            for component in out_degree[itteration - 1]:
                  for out_deg in component:
                        out_degree2.append(out_deg)

           #for weighted degree       
            weighted_degree2=[]   
            for component in weightd_degree[itteration - 1]:
                  for weight_deg in component:
                        weighted_degree2.append(weight_deg)             

            df = pd.DataFrame()

            df.insert(loc=0,
                      column='patients_id',
                      value=self.vertices)
          
            df.insert(loc=1,
                      column='community_id',
                      value=id)

            df.insert(loc=2,
                      column='in_degree',
                      value=weightt)
            df.insert(loc=3,
                        column='out_degree',
                        value=out_degree2)

            df.insert(loc=4,
                        column='weighted_degree',
                       value=weighted_degree2)
     
            return(df)

        if itteration2!=-1:
            #for id
            # idd = 1
            # id = []
            # for component in self.pi6[itteration - 1]:
                # for vertex in component:
                    # id.append(idd)
                # idd = idd + 1
                
            self.vertices = []
            idd = 1
            id = []
            age=[]
            gender=[]
            primary=[]
            fsa=[]
            n_encounters=[]
            n_interactions=[]
            first_admit_dt=[]
            last_admit_dt=[]
            covid_positive=[]
         
            
            for component in self.pi6[itteration - 1]:
                for vertex in component:
                    v= self.map4.get(vertex)
                    self.vertices.append(v)
                    '''index = np.where(self.patients.patient_id== v)[0]'''
                    id.append(idd)
                idd = idd + 1 

            #for weight
            weightt=[]
            for component in weight[itteration - 1]:
                for degree in component:
                    weightt.append(degree)
            df = pd.DataFrame()                   
                    
                    

            #for out degree       
            out_degree2=[]   
            for component in out_degree[itteration - 1]:
                  for out_deg in component:
                        out_degree2.append(out_deg)

            #for weighted degree       
            weighted_degree2=[]   
            for component in weightd_degree[itteration - 1]:
                 for weight_deg in component:
                        weighted_degree2.append(weight_deg)            

                        
                        

            df = pd.DataFrame()

            df.insert(loc=0,
                      column='patients_id',
                      value=self.vertices)

            
            df.insert(loc=1,
                      column='community_id',
                      value=id)

            df.insert(loc=2,
                      column='in_degree',
                      value=weightt)
            df.insert(loc=3,
                        column='out_degree',
                        value=out_degree2)
            


            df.insert(loc=4,
                        column='weighted_degree',
                       value=weighted_degree2)
            
            

            
            return(df)
        
        
    def count_nodes_in_communities(self, iteration=-1):
        iteration = iteration if iteration != -1 else len(self.pi6)
        nodes_counts = {}
        for i in range(iteration):
            communities = self.pi6[i]
            for j, community in enumerate(communities):
                community_id = f"Community {j+1} (Iteration {i+1})"
                nodes_counts[community_id] = len(community)
        return nodes_counts


custom_iteration1 = int(input("Enter the number of iterations:"))

patient_community_analysis = PatientCommunityAnalysis(engine)
interactions_df = patient_community_analysis.interactions()


patient_community_analysis.create_graph(interactions_df)

communities, graph = patient_community_analysis.community_detection_on_patients(number_of_iterations=custom_iteration1)
output_file_path = 'Iteration_Information_Fall.txt'
with open(output_file_path, 'w') as file:
    file.write("done!\n")
    file.write(f"Number of all iterations: {len(communities)}\n")
    for iteration_num, community_list in enumerate(communities, start=1):
        file.write(f"Number of communities in iteration {iteration_num}: {len(community_list)}\n")

custom_iteration2 = int(input("Enter your desired iteration:"))
patient_community = patient_community_analysis.create_table_for_patients(iteration2=custom_iteration2)
patient_community.to_csv('Patient_Community_Projection_Fall.csv', index=False)
print(patient_community)

nodes_counts = patient_community_analysis.count_nodes_in_communities(iteration=custom_iteration2)
for community_id, node_count in nodes_counts.items():
    print(f"{community_id}: {node_count}")